In [ ]:
# CPU Computing
import tensorflow as tf
devices=tf.config.list_physical_devices()
tf.config.set_visible_devices([],'GPU')
tf.config.set_visible_devices(devices[0],'CPU')

In [ ]:
tf.config.get_visible_devices()

In [ ]:
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from scipy import stats
from scipy import integrate
import warnings

warnings.filterwarnings('error')

In [ ]:
market=pd.read_csv('market.csv')
stock=pd.read_csv('stock.csv')

In [ ]:
# stock return from unit to %
stock['rret']=stock['rret']*100
# mktret return 
market['mktret']=market['mktrf']+market['rf']

In [ ]:
market['date']=pd.to_datetime(market['date'])
stock['date']=pd.to_datetime(stock['date'])

In [ ]:
stock=stock[stock['date']>='1995-01']
market=market[market['date']>='1995-01']

In [ ]:
# drop stocks traded less than 30 days
stock_list=[]
for i,group in stock.groupby('permno'):
    if group.shape[0]<30:
        continue
    else:
        stock_list.append(group)
stock=pd.concat(stock_list,axis=0)
stock.dropna(how='any',axis=0,inplace=True)
stock.reset_index(drop=True,inplace=True)

In [ ]:
# mark the last day of the month 
monthly_stock_list=[]
for i,group in stock.groupby('permno'):
    group=group.resample('Q',on='date').last()
    monthly_stock_list.append(group)
monthly_stock=pd.concat(monthly_stock_list,axis=0)
monthly_stock.dropna(how='any',axis=0,inplace=True)
monthly_stock.reset_index(drop=True,inplace=True)

In [ ]:
# find the last day in the daily sample
stock['mark']=0
stock_list=[]
for (i,group),(monthly_i,monthly_group) in zip(stock.groupby('permno'),monthly_stock.groupby('permno')):
    judge=monthly_group['date'].unique()
    group.loc[group['date'].isin(judge),'mark']=1
    stock_list.append(group)
stock=pd.concat(stock_list,axis=0)

In [ ]:
# empirical CDF
def CDF(x):
    x_sorted=x.sort_values()

    index=x_sorted.index
    n=np.arange(len(x_sorted))
    n=n+1
    n=pd.Series(n,index=index,name='cdf_series')

    cdf_series=n/(len(x_sorted)+1)

    x=pd.concat([x,cdf_series],axis=1)
    return x['cdf_series']

In [ ]:
cdf_market=CDF(market['mktret'])
cdf_stock=CDF(stock['rret'])

In [ ]:
market=pd.concat([market,cdf_market],axis=1)
stock=pd.concat([stock,cdf_stock],axis=1)

In [ ]:
# merge factor to stock
df=pd.merge(stock,market,on='date',how='left')
df.rename(columns={'cdf_series_x':'cdf_stock','cdf_series_y':'cdf_market'},inplace=True)

In [ ]:
# save data
df.to_csv('data.csv',index=False)

In [ ]:
# read data
df=pd.read_csv('data.csv')

In [ ]:
def group_by_train(group):
    import tensorflow as tf
    devices=tf.config.list_physical_devices()
    tf.config.set_visible_devices([],'GPU')
    tf.config.set_visible_devices(devices[0],'CPU')

    import numpy as np
    import pandas as pd
    from scipy import stats
    from scipy import integrate


    # Clayton Copula lower tail dependence 
    # theta>=0
    def Clayton_Copula_CDF(theta,u1,u2):
        if theta==0:
            return u1*u2
        else:
            base=tf.math.pow(u1,-theta)+tf.math.pow(u2,-theta)-1
            return tf.math.pow(base,-1/theta)

    def Clayton_Copula_PDF(theta,u1,u2):
        return (theta + 1)/(u1**(theta + 1)*u2**(theta + 1)*(1/u1**theta + 1/u2**theta - 1)**(1/theta + 2))


    # Caussian_Copula no tail dependence 
    # -1<=theta<=1
    def Gaussian_Copula_CDF(theta,u1,u2):
        u1_inv=stats.norm.ppf(u1)
        u2_inv=stats.norm.ppf(u2)

        def f(x,y,theta):
            return 1/(2*np.pi*tf.math.sqrt(1-theta**2))*tf.math.pow(np.e,(-y*y+2*theta*x*y-x*x)/(2*(1-theta**2)))

        def cdf(theta,u1_inv,u2_inv):
            return integrate.dblquad(f,-np.inf,u1_inv,-np.inf,u2_inv,args=(theta,))

        return cdf(theta,u1_inv,u2_inv)[0]

    def Gaussian_Copula_PDF(theta,u1,u2):
        u1_inv=stats.norm.ppf(u1)
        u2_inv=stats.norm.ppf(u2)

        def f(x,y,theta):
            return 1/(2*np.pi*tf.math.sqrt(1-theta**2))*tf.math.exp((-x*x+2*theta*x*y-y*y)/(2*(1-theta**2)))

        return 1/stats.norm.pdf(u1)*1/stats.norm.pdf(u2)*f(u1_inv,u2_inv,theta)


    # Gumbel_Copula Copula upper tail dependence 
    # theta>=1
    def Gumbel_Copula_CDF(theta,u1,u2):
        m1=-tf.math.log(u1)
        m2=-tf.math.log(u2)
        return tf.math.exp(-tf.math.pow(m1**theta+m2**theta,1/theta))

    def Gumbel_Copula_PDF(theta,u1,u2):
        return (tf.math.exp(-((-tf.math.log(u1))**theta + (-tf.math.log(u2))**theta)**(1/theta))*(-tf.math.log(u1))**(theta - 1)*((-tf.math.log(u1))**theta + (-tf.math.log(u2))**theta)**(1/theta - 1))/u1

    #combine copulas
    def combine_Copula(w1,w2,w3,Copula_1,Copula_2,Copula_3):
        return w1*Copula_1+w2*Copula_2+w3*Copula_3
        
    def mle(theta1,theta2,theta3,w1,w2,w3,u1,u2):
        Copula_1=Clayton_Copula_PDF(theta1,u1,u2)
        Copula_2=Gaussian_Copula_PDF(theta2,u1,u2)
        Copula_3=Gumbel_Copula_PDF(theta3,u1,u2)
        
        prop_series=combine_Copula(w1,w2,w3,Copula_1,Copula_2,Copula_3)
        prop_series=tf.math.log(prop_series)
        result=tf.reduce_sum(prop_series)
        return -result
    
    # theta>=0
    def dClayton_du(theta,u1,u2):
        return 1/(u1**(theta+1)*(1/u1**theta+1/u2**theta-1)**(1/theta+1))

    # -1<=theta<=1
    def dGaussian_du(theta,u1,u2):
        u1_inv=stats.norm.ppf(u1)
        u2_inv=stats.norm.ppf(u2)

        def f(x,y,theta):
            return 1/(2*np.pi*tf.math.sqrt(1-theta**2))*tf.math.pow(np.e,(-y*y+2*theta*x*y-x*x)/(2*(1-theta**2)))

        def derivative(theta,u1_inv,u2_inv):
            result=integrate.quad(f,-np.inf,u1_inv,args=(u2_inv,theta))
            return result[0]

        return 1/stats.norm.pdf(u2)*derivative(theta,u1_inv,u2_inv)

    # theta>=1
    def dGumbel_du(theta,u1,u2):
        return (np.exp(-((-np.log(u1))**theta+(-np.log(u2))**theta)*(1/theta))*(-np.log(u1))**(theta-1)*((-np.log(u1))**theta+(-np.log(u2))**theta)**(1/theta-1))/u1

    class GreaterEqualZero(tf.keras.constraints.Constraint):
        def __call__(self,value):
            value=tf.math.maximum(value,0)
            return value

    class MinusOneToOne(tf.keras.constraints.Constraint):
        def __call__(self,value):
            value=tf.math.maximum(value,-1)
            value=tf.math.minimum(value,1)
            return value

    class ZeroToOne(tf.keras.constraints.Constraint):
        def __call__(self,value):
            value=tf.math.maximum(value,0)
            value=tf.math.minimum(value,1)
            return value

    class GreaterEqualOne(tf.keras.constraints.Constraint):
        def __call__(self,value):
            value=tf.math.maximum(value,1)
            return value

    cons1=GreaterEqualZero()
    cons2=MinusOneToOne()
    cons3=GreaterEqualOne()
    cons4=ZeroToOne()


    first_index=group.head(1).index[0]
    adam_attempt=0
    adam_decay_rate_list=np.concatenate([np.arange(0.000,0.020,0.002),np.arange(0.02,0.20,0.02),np.arange(0.2,2.0,0.1),np.arange(2,20,1),np.arange(20,100,10),np.arange(100,1000,100)],axis=0)
    
    # define window CPU training
    def window_train(window):   
        nonlocal adam_attempt
        try:
            # construct maximum likelihood function 
            theta1=tf.Variable(np.random.uniform(0,1),name='theta1',dtype='float64',constraint=cons1)
            theta2=tf.Variable(np.random.uniform(-1,1),name='theta2',dtype='float64',constraint=cons2)
            theta3=tf.Variable(np.random.uniform(1,2),name='theta3',dtype='float64',constraint=cons3)
            w1=tf.Variable(np.random.uniform(0,0.5),name='w1',dtype='float64',constraint=cons4)
            w2=tf.Variable(np.random.uniform(0,0.5),name='w2',dtype='float64',constraint=cons4)
            w3=tf.Variable(np.random.uniform(0,0.5),name='w3',dtype='float64',constraint=cons4)
            beta=tf.constant(1e5,name='beta',dtype='float64')

            cdf_stock=tf.convert_to_tensor(window['cdf_stock'].to_numpy(),dtype='float64')
            cdf_market=tf.convert_to_tensor(window['cdf_market'].to_numpy(),dtype='float64')
            train=tf.data.Dataset.from_tensor_slices((cdf_stock,cdf_market))
            train=train.batch(window.shape[0])
        
            mle_list=[]
            optimizer=tf.optimizers.Adam(learning_rate=0.001,decay=adam_decay_rate_list[adam_attempt])
            result_list=[]
            for epoch in range(2000):
                for cdf_stock,cdf_market in train:
                    with tf.GradientTape(persistent=True) as tape:
                        result=(
                            mle(theta1,theta2,theta3,w1,w2,w3,cdf_stock,cdf_market)+
                            beta*(w1+w2+w3-1)**2
                        )
                    result_list.append(result)
                    gradients=tf.clip_by_norm(tape.gradient(result,(theta1,theta2,theta3,w1,w2,w3)),1)

                    optimizer.apply_gradients(zip(gradients,(theta1,theta2,theta3,w1,w2,w3)))  
                    if np.isnan(result):
                        raise
                
            # LTD and UTD
            theta1,theta2,theta3,w1,w2,w3=np.round([theta1,theta2,theta3,w1,w2,w3],4)

            u_low=tf.constant(1e-300,dtype='float64')
            Copula_1_LTD=Clayton_Copula_CDF(theta1,u_low,u_low)/u_low
            LTD=np.round(w1*Copula_1_LTD,4)

            u_high=tf.constant(1-1e-16,dtype='float64')
            Copula_3_UTD=2-dGumbel_du(theta3,u_high,u_high)
            UTD=np.round(w3*Copula_3_UTD,4)

            trained=1

            adam_attempt=adam_attempt+1

        except:
            adam_attempt=adam_attempt+1
            if adam_attempt<adam_decay_rate_list.shape[0]:
                trained,LTD,UTD=window_train(window)
            else:
                trained,LTD,UTD=1,np.nan,np.nan 
        return trained,LTD,UTD
    
    
    # apply row train
    def row_by_window(row,group):
        nonlocal first_index
        index=row.name
        mark=row['mark']

        if mark==0:
            return 0,np.nan,np.nan
        elif (mark==1) & (index-251<first_index):
            return 0,np.nan,np.nan
        else:
            window=group.loc[index-251:index,:]
            trained,LTD,UTD=window_train(window)
            return trained,LTD,UTD
    
    train=group.apply(row_by_window,axis=1,result_type='expand',args=(group,))
    train.columns=['trained','LTD','UTD']
    group=pd.concat([group,train],axis=1)
    group=group[(group['mark']==1)&(group['trained']==1)]
    return group

In [ ]:
pandarallel.initialize(nb_workers=60)
final=df.groupby('permno').parallel_apply(group_by_train)